In [44]:

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms


In [45]:
import pandas as pd
file_path = '/content/labels.csv'
print(f"Attempting to read from: {file_path}")
df = pd.read_csv(file_path, header=None)
df.columns = ['filename','class','xmin','ymin','xmax','ymax']
df.head()

Attempting to read from: /content/labels.csv


,filename,class,xmin,ymin,xmax,ymax
0,0,pickup_truck,213,34,255,50
1,0,car,194,78,273,122
2,0,car,155,27,183,35
3,0,articulated_truck,43,25,109,55
4,0,car,106,32,124,45


In [46]:
import os

file_path = '/content/labels.csv'

if os.path.exists(file_path):
    try:
        with open(file_path, 'r') as f:
            print(f"Successfully opened {file_path}")
            # Read and print the first few lines
            for i in range(5):
                line = f.readline()
                if not line:
                    break
                print(f"Line {i+1}: {line.strip()}")
    except Exception as e:
        print(f"An error occurred while trying to read the file: {e}")
else:
    print(f"Error: File not found at {file_path}")

Successfully opened /content/labels.csv
Line 1: 00000000,pickup_truck,213,34,255,50
Line 2: 00000000,car,194,78,273,122
Line 3: 00000000,car,155,27,183,35
Line 4: 00000000,articulated_truck,43,25,109,55
Line 5: 00000000,car,106,32,124,45


In [47]:
import os
print(os.listdir('/content/'))

['.config', 'Images.zip', 'labels.csv', 'Images', 'Images (1).zip', 'sample_data']


In [48]:
df = df[['filename','xmin','ymin','xmax','ymax','class']]

In [49]:
from sklearn.model_selection import train_test_split

images = df['filename'].unique()
train_imgs, test_imgs = train_test_split(images, test_size=0.2, random_state=42)
print("Train:", len(train_imgs), "Test:", len(test_imgs))

Train: 88000 Test: 22000


In [50]:
image_dir = "/content/Images"

In [51]:
df['filename'] = df['filename'].astype(str) + ".jpg"

In [52]:
df['filename'] = (
    df['filename']
    .astype(str)               # force to string
    .str.extract('(\d+)')[0]   # keep only digits
    .astype(int)               # convert to integer
    .astype(str)
    .str.zfill(8) + '.jpg'     # pad and add extension
)

<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipython-input-2080775916.py:4: SyntaxWarning: invalid escape sequence '\d'
  .str.extract('(\d+)')[0]   # keep only digits


In [53]:
print(df['filename'].head())

0    00000000.jpg
1    00000000.jpg
2    00000000.jpg
3    00000000.jpg
4    00000000.jpg
Name: filename, dtype: object


In [54]:
class VehicleDataset(Dataset):
    def __init__(self, df, image_dir, image_list, transform=None):
        self.df = df
        self.image_dir = image_dir
        # make sure filenames are strings
        self.image_list = [str(x) for x in image_list]
        self.transform = transform

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        img_name = str(self.image_list[idx])
        img_path = os.path.join(self.image_dir, img_name)
        image = Image.open(img_path).convert("RGB")

        boxes_df = self.df[self.df['filename'] == img_name]
        boxes = boxes_df[['xmin', 'ymin', 'xmax', 'ymax']].values
        labels = boxes_df['class'].astype('category').cat.codes.values

        if self.transform:
            image = self.transform(image)

        target = {
            'boxes': torch.as_tensor(boxes, dtype=torch.float32),
            'labels': torch.as_tensor(labels, dtype=torch.int64)
        }

        return image, target


In [55]:
transform = transforms.Compose([transforms.ToTensor()])

train_dataset = VehicleDataset(df, image_dir, train_imgs, transform)
test_dataset = VehicleDataset(df, image_dir, test_imgs, transform)

train_loader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    collate_fn=lambda x: tuple(zip(*x))
)


In [56]:
from torchvision import transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([transforms.ToTensor()])

train_dataset = VehicleDataset(df, image_dir, train_imgs, transform)
test_dataset = VehicleDataset(df, image_dir, test_imgs, transform)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))


In [57]:
missing = []
for img in df['filename'].unique():
    path = os.path.join(image_dir, img)
    if not os.path.exists(path):
        missing.append(img)
print("Missing images:", len(missing))


Missing images: 110000


In [58]:
!rm /content/Images.zip
from google.colab import files
uploaded = files.upload()


Saving Images.zip to Images.zip


In [60]:
import zipfile, os

zip_path = '/content/Images.zip'
extract_path = '/content/Images'

if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("Unzipped successfully:", extract_path)
    print("Sample files:", os.listdir(extract_path)[:10])
else:
    print("File not found.")


File not found.


In [64]:
zip_path = '/content/Images (1).zip'
extract_path = '/content/Images'

import zipfile, os

if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"✅ '{zip_path}' unzipped to '{extract_path}'")
    print("Extracted files:", os.listdir(extract_path))
else:
    print(f"❌ Error: '{zip_path}' not found.")


✅ '/content/Images (1).zip' unzipped to '/content/Images'
Extracted files: ['Images']


In [66]:
import os
print(os.listdir('/content/'))

['.config', 'labels.csv', 'Images', 'Images (1).zip', 'sample_data']


In [67]:
missing = [f for f in df['filename'].unique() if not os.path.exists(os.path.join(image_dir, f))]
print("Missing images:", len(missing))


Missing images: 110000


In [68]:
missing = [f for f in df['filename'].unique() if not os.path.exists(os.path.join(image_dir, f))]
print("Missing images:", len(missing))


Missing images: 110000


In [69]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN_ResNet50_FPN_Weights

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)

In [70]:
num_classes = len(df['class'].unique()) + 1  # +1 for background
print("Classes:", num_classes)

Classes: 12


In [71]:
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)


In [72]:
import os

df = df[df['filename'].apply(lambda x: os.path.exists(os.path.join(image_dir, x)))]
print("Remaining images after dropping missing ones:", len(df))

Remaining images after dropping missing ones: 0


In [73]:
import os

df = df[df['filename'].apply(lambda x: os.path.exists(os.path.join(image_dir, x)))]
print("Remaining images after dropping missing ones:", len(df))

Remaining images after dropping missing ones: 0


In [74]:
train_dataset = VehicleDataset(df, image_dir, train_imgs, transform)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))


In [77]:


df = pd.read_csv('/content/labels.csv', header=None)
print(df.head())


   0                  1    2   3    4    5
0  0       pickup_truck  213  34  255   50
1  0                car  194  78  273  122
2  0                car  155  27  183   35
3  0  articulated_truck   43  25  109   55
4  0                car  106  32  124   45


In [79]:
import os
print(os.listdir('/content'))


['.config', 'labels.csv', 'Images', 'Images (1).zip', 'sample_data']


In [81]:
# 1. Show what’s inside the extracted folder
import os
print(os.listdir('/content/Images')[:10])


['Images']


In [82]:
# 2. Show what the first few filenames look like in your CSV
import pandas as pd
df = pd.read_csv('/content/labels.csv', header=0)
print(df.head())
print(df.columns)


   00000000       pickup_truck  213   34  255   50
0         0                car  194   78  273  122
1         0                car  155   27  183   35
2         0  articulated_truck   43   25  109   55
3         0                car  106   32  124   45
4         1                bus  205  155  568  314
Index(['00000000', 'pickup_truck', '213', '34', '255', '50'], dtype='object')


In [83]:
# 3. Compare CSV filenames to image files
csv_files = set(df[df.columns[0]].astype(str))
img_files = set(os.listdir('/content/Images'))
missing = csv_files - img_files
print("Sample missing files:", list(missing)[:5])


Sample missing files: ['54290', '2844', '58450', '1127', '9888']


In [86]:
# Ensure filenames have the .jpg extension if your images are named like '00000000.jpg'
df['filename'] = df[df.columns[0]].astype(str) + '.jpg'
print(df['filename'].head())


0    0.jpg
1    0.jpg
2    0.jpg
3    0.jpg
4    1.jpg
Name: filename, dtype: object


In [87]:
filename_col = 'filename'


In [91]:
# Add .jpg (or .png) to all filenames depending on your actual files
df['filename'] = df[df.columns[0]].astype(str) + '.jpg'

# Then set this column as the identifier
filename_col = 'filename'

# Verify alignment
print(df['filename'].head())
print(os.listdir('/content/Images')[:5])


0    0.jpg
1    0.jpg
2    0.jpg
3    0.jpg
4    1.jpg
Name: filename, dtype: object
['Images']


In [94]:
import os
print(os.listdir('/content'))


['.config', 'labels.csv', 'Images', 'Images (1).zip', 'sample_data']


In [102]:
# use the correct label column from your CSV
num_classes = len(df['pickup_truck'].unique()) + 1


In [104]:
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn
import pandas as pd

# reload CSV to confirm columns
df = pd.read_csv('/content/labels.csv')
print("Columns:", df.columns)

# choose the right label column from your CSV
label_col = df.columns[1]   # e.g., 'pickup_truck' if that’s what appears
num_classes = len(df[label_col].unique()) + 1
print("Detected label column:", label_col)
print("num_classes =", num_classes)

# load model
model = fasterrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


Columns: Index(['00000000', 'pickup_truck', '213', '34', '255', '50'], dtype='object')
Detected label column: pickup_truck
num_classes = 12


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [106]:
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import pandas as pd

# === Reload labels.csv and inspect columns ===
df = pd.read_csv('/content/labels.csv')
print("Columns found:", list(df.columns))

# Automatically pick a usable label column
possible_cols = [c for c in df.columns if c.lower() not in ['filename', 'id', 'image', 'x', 'y']]
if len(possible_cols) == 0:
    raise ValueError("No valid label column found. Check your CSV headers.")
label_col = possible_cols[0]
print("Detected label column:", label_col)

# === Determine class count ===
num_classes = len(df[label_col].unique()) + 1
print("num_classes =", num_classes)

# === Load pretrained Faster R-CNN and reconfigure for dataset ===
model = fasterrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print("✅ Model ready on", device)


Columns found: ['00000000', 'pickup_truck', '213', '34', '255', '50']
Detected label column: 00000000
num_classes = 110001


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


✅ Model ready on cuda


In [108]:
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import pandas as pd

# === Reload CSV and inspect ===
df = pd.read_csv('/content/labels.csv')
print("Columns in CSV:", list(df.columns))

# === Automatically find the label column ===
label_candidates = [c for c in df.columns if c.lower() not in ['filename', 'id', 'image', 'x', 'y']]
if not label_candidates:
    raise ValueError("No label column found in labels.csv.")
label_col = label_candidates[0]
print("Detected label column:", label_col)

# === Define number of classes ===
num_classes = len(df[label_col].unique()) + 1
print("Number of classes:", num_classes)

# === Load pretrained model ===
model = fasterrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print("✅ Model loaded successfully on", device)


Columns in CSV: ['00000000', 'pickup_truck', '213', '34', '255', '50']
Detected label column: 00000000
Number of classes: 110001


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


✅ Model loaded successfully on cuda


In [109]:
from torch.optim import SGD

params = [p for p in model.parameters() if p.requires_grad]
optimizer = SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

num_epochs = 5


In [110]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [120]:
import os
print(os.listdir('/content'))


['.config', 'Images.zip', '.ipynb_checkpoints', 'labels.csv', 'Images', 'vehicle_detector.pth', 'sample_data']


In [126]:
import os, zipfile, shutil
from google.colab import files

# === 1. RESET ENVIRONMENT ===
for p in ["/content/Images", "/content/Images.zip", "/content/Images (1).zip"]:
    if os.path.exists(p):
        if os.path.isdir(p):
            shutil.rmtree(p)
        else:
            os.remove(p)
print("🧹 Clean environment ready.")

# === 2. UPLOAD ZIP ===
print("📤 Upload your real Images.zip (compressed properly). Wait for 100% complete...")
uploaded = files.upload()
fname = list(uploaded.keys())[0]
print("Uploaded:", fname)

# Normalize filename
os.rename(f"/content/{fname}", "/content/Images.zip")
zip_path = "/content/Images.zip"

# === 3. AUTO-REPAIR CHECK ===
def is_valid_zip(path):
    try:
        with zipfile.ZipFile(path, "r") as z:
            bad = z.testzip()
            if bad:
                print("❌ Bad file inside ZIP:", bad)
                return False
            return True
    except Exception as e:
        print("❌ Not a valid ZIP:", e)
        return False

if not is_valid_zip(zip_path):
    print("\n⚙️ Attempting to fix ZIP structure...")
    fixed_path = "/content/Images_fixed.zip"
    with open(zip_path, "rb") as f:
        data = f.read()
    if b"PK" not in data[:4]:
        raise ValueError("❌ This file is not a real ZIP. Recompress your folder correctly and re-upload.")
    with open(fixed_path, "wb") as f:
        f.write(data[data.find(b"PK"):])
    os.remove(zip_path)
    os.rename(fixed_path, zip_path)
    assert is_valid_zip(zip_path), "❌ Still invalid after repair."

print("✅ ZIP verified as valid.")

# === 4. EXTRACT ===
extract_path = "/content/Images"
with zipfile.ZipFile(zip_path, "r") as z:
    z.extractall(extract_path)

# Handle nested folder case
nested = os.path.join(extract_path, "Images")
if os.path.exists(nested):
    extract_path = nested

print(f"✅ Extracted successfully to: {extract_path}")
print("Sample files:", os.listdir(extract_path)[:10])


🧹 Clean environment ready.
📤 Upload your real Images.zip (compressed properly). Wait for 100% complete...


Saving Images 2.zip to Images 2 (1).zip
Uploaded: Images 2 (1).zip
✅ ZIP verified as valid.
✅ Extracted successfully to: /content/Images/Images
Sample files: ['00003433.jpg', '00004110.jpg', '00003593.jpg', '00002457.jpg', '00002487.jpg', '00000728.jpg', '00003036.jpg', '00000307.jpg', '00001057.jpg', '00004164.jpg']


In [132]:
import os
import pandas as pd

# Read and inspect your CSV
df = pd.read_csv('/content/labels.csv')
print("CSV columns:", df.columns.tolist())
print(df.head())

# Check what image names actually exist
files = os.listdir('/content/Images')[:10]
print("Sample image files:", files)


CSV columns: ['00000000', 'pickup_truck', '213', '34', '255', '50']
   00000000       pickup_truck  213   34  255   50
0         0                car  194   78  273  122
1         0                car  155   27  183   35
2         0  articulated_truck   43   25  109   55
3         0                car  106   32  124   45
4         1                bus  205  155  568  314
Sample image files: ['Images', '__MACOSX']


In [133]:
import os, shutil

base_path = '/content/Images'
nested_path = os.path.join(base_path, 'Images')

# Move files out if nested
if os.path.exists(nested_path):
    for f in os.listdir(nested_path):
        shutil.move(os.path.join(nested_path, f), base_path)
    shutil.rmtree(nested_path)
    print("✅ Fixed nested folder. All images moved to /content/Images")

# Remove MacOS junk folder
macosx_path = os.path.join(base_path, '__MACOSX')
if os.path.exists(macosx_path):
    shutil.rmtree(macosx_path)
    print("🧹 Removed __MACOSX folder.")

print("Sample after fix:", os.listdir(base_path)[:10])


✅ Fixed nested folder. All images moved to /content/Images
🧹 Removed __MACOSX folder.
Sample after fix: ['00003433.jpg', '00004110.jpg', '00003593.jpg', '00002457.jpg', '00002487.jpg', '00000728.jpg', '00003036.jpg', '00000307.jpg', '00001057.jpg', '00004164.jpg']


In [134]:
print(os.listdir('/content/Images')[:10])


['00003433.jpg', '00004110.jpg', '00003593.jpg', '00002457.jpg', '00002487.jpg', '00000728.jpg', '00003036.jpg', '00000307.jpg', '00001057.jpg', '00004164.jpg']


In [135]:
df['filename'] = df[df.columns[0]].astype(str) + '.jpg'
existing_files = set(os.listdir('/content/Images'))
valid_df = df[df['filename'].isin(existing_files)]
print(f"✅ Fixed valid entries: {len(valid_df)} / {len(df)}")


✅ Fixed valid entries: 0 / 351548


In [138]:
import os
import pandas as pd

# === 1. Inspect image folder ===
img_dir = '/content/Images'
img_files = sorted(os.listdir(img_dir))
print("Sample image files:", img_files[:10])

# === 2. Inspect CSV ===
df = pd.read_csv('/content/labels.csv')
print("\nCSV columns:", df.columns.tolist())
print(df.head(5))

# === 3. Auto-match logic ===
# Get first column name
col = df.columns[0]
print("\nDetected filename column:", col)

# Build multiple possible filename formats
df['base'] = df[col].astype(str)
formats = {
    'raw': df['base'],
    'jpg': df['base'] + '.jpg',
    'png': df['base'] + '.png',
    'zero8_jpg': df['base'].str.zfill(8) + '.jpg',
    'zero5_jpg': df['base'].str.zfill(5) + '.jpg'
}

# === 4. Test each format ===
existing = set(img_files)
match_counts = {k: sum(f.isin(existing)) for k, f in formats.items()}
print("\nMatch counts per format:", match_counts)

# === 5. Pick best match automatically ===
best_fmt = max(match_counts, key=match_counts.get)
df['filename'] = formats[best_fmt]
valid_df = df[df['filename'].isin(existing)]

print(f"\n✅ Best match format: '{best_fmt}'")
print(f"✅ Valid entries: {len(valid_df)} / {len(df)}")
print("Sample matched filenames:", valid_df['filename'].head(10).tolist())


Sample image files: ['00000000.jpg', '00000001.jpg', '00000002.jpg', '00000003.jpg', '00000004.jpg', '00000005.jpg', '00000006.jpg', '00000007.jpg', '00000008.jpg', '00000009.jpg']

CSV columns: ['00000000', 'pickup_truck', '213', '34', '255', '50']
   00000000       pickup_truck  213   34  255   50
0         0                car  194   78  273  122
1         0                car  155   27  183   35
2         0  articulated_truck   43   25  109   55
3         0                car  106   32  124   45
4         1                bus  205  155  568  314

Detected filename column: 00000000

Match counts per format: {'raw': 0, 'jpg': 0, 'png': 0, 'zero8_jpg': 17966, 'zero5_jpg': 0}

✅ Best match format: 'zero8_jpg'
✅ Valid entries: 17966 / 351548
Sample matched filenames: ['00000000.jpg', '00000000.jpg', '00000000.jpg', '00000000.jpg', '00000001.jpg', '00000001.jpg', '00000001.jpg', '00000001.jpg', '00000001.jpg', '00000001.jpg']


In [139]:
train_imgs, test_imgs = train_test_split(valid_df['filename'].unique(), test_size=0.2, random_state=42)


In [143]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image, UnidentifiedImageError
import os
import numpy as np

class VehicleDataset(Dataset):
    def __init__(self, df, image_dir, file_column='filename', transform=None):
        self.df = df.reset_index(drop=True)
        self.image_dir = image_dir
        self.file_column = file_column
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.image_dir, row[self.file_column])

        # Handle missing/corrupted images safely
        try:
            img = Image.open(img_path).convert("RGB")
        except (FileNotFoundError, UnidentifiedImageError):
            # Return a black image instead of crashing
            img = Image.fromarray(np.zeros((224, 224, 3), dtype=np.uint8))

        # Convert non-numeric labels to tensor
        label_data = row.drop(self.file_column).values
        label_data = [float(x) if str(x).replace('.', '', 1).isdigit() else 0.0 for x in label_data]
        target = torch.tensor(label_data, dtype=torch.float32)

        if self.transform:
            img = self.transform(img)

        return img, target


# === FIXED DATALOADER SECTION ===
from sklearn.model_selection import train_test_split

train_imgs, test_imgs = train_test_split(valid_df['filename'].unique(), test_size=0.2, random_state=42)
train_df = valid_df[valid_df['filename'].isin(train_imgs)]
test_df = valid_df[valid_df['filename'].isin(test_imgs)]

train_dataset = VehicleDataset(train_df, '/content/Images', transform=transform)
test_dataset = VehicleDataset(test_df, '/content/Images', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=0)


In [146]:
from torchvision import transforms

# Standardize all images to same size
transform = transforms.Compose([
    transforms.Resize((224, 224)),   # ensures equal dimensions
    transforms.ToTensor(),           # converts to tensor
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])  # ImageNet normalization
])


In [148]:
  train_dataset = VehicleDataset(train_df, '/content/Images', transform=transform)
test_dataset = VehicleDataset(test_df, '/content/Images', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=8, num_workers=0)


In [150]:
import torch

def format_target(row):
    # Convert CSV bounding box and class data to PyTorch dict
    try:
        boxes = torch.tensor([[float(row['xmin']), float(row['ymin']),
                               float(row['xmax']), float(row['ymax'])]],
                             dtype=torch.float32)
        labels = torch.tensor([int(row['class_id'])], dtype=torch.int64)
        return {"boxes": boxes, "labels": labels}
    except Exception as e:
        print(f"⚠️ Skipped faulty row: {e}")
        return None


In [154]:
# STEP 2 — LOAD LABELS CSV (MANUAL COLUMN FIX)
csv_path = "/content/labels.csv"
df = pd.read_csv(csv_path)
print("📘 CSV columns:", df.columns.tolist())
print(df.head())

# Manually rename the first column to 'filename'
df = df.rename(columns={df.columns[0]: 'filename'})
print("✅ Renamed first column as 'filename'")


📘 CSV columns: ['00000000', 'pickup_truck', '213', '34', '255', '50']
   00000000       pickup_truck  213   34  255   50
0         0                car  194   78  273  122
1         0                car  155   27  183   35
2         0  articulated_truck   43   25  109   55
3         0                car  106   32  124   45
4         1                bus  205  155  568  314
✅ Renamed first column as 'filename'


In [156]:
# STEP 2 — LOAD + FIX CSV (MATCHES "00003433.jpg" STYLE FILENAMES)
csv_path = "/content/labels.csv"
df = pd.read_csv(csv_path)
print("📘 CSV columns:", df.columns.tolist())
print(df.head())

# Rename first column to 'filename'
df = df.rename(columns={df.columns[0]: "filename"})
df["filename"] = df["filename"].astype(str)

# Match naming patterns like zero-padded filenames
image_dir = "/content/Images"
existing = os.listdir(image_dir)
jpg_files = [f for f in existing if f.lower().endswith(".jpg")]

# Try different zero-padding formats (like 6, 7, 8 digits)
formats = [lambda x: f"{int(x):06d}.jpg", lambda x: f"{int(x):07d}.jpg", lambda x: f"{int(x):08d}.jpg"]
best_fmt = None
max_matches = 0

for fmt in formats:
    temp_names = df["filename"].apply(lambda x: fmt(x) if x.isdigit() else x)
    matches = sum(temp_names.isin(jpg_files))
    if matches > max_matches:
        max_matches = matches
        best_fmt = fmt

df["filename"] = df["filename"].apply(lambda x: best_fmt(x) if x.isdigit() else x)
df = df[df["filename"].isin(jpg_files)]

print(f"✅ Matched format detected. Valid entries: {len(df)}")
print("📸 Sample matched filenames:", df['filename'].head().tolist())


📘 CSV columns: ['00000000', 'pickup_truck', '213', '34', '255', '50']
   00000000       pickup_truck  213   34  255   50
0         0                car  194   78  273  122
1         0                car  155   27  183   35
2         0  articulated_truck   43   25  109   55
3         0                car  106   32  124   45
4         1                bus  205  155  568  314
✅ Matched format detected. Valid entries: 17966
📸 Sample matched filenames: ['00000000.jpg', '00000000.jpg', '00000000.jpg', '00000000.jpg', '00000001.jpg']


In [159]:
# ==============================================================
# FIXED VEHICLE DATASET (NO SERIES / BAD IMAGES)
# ==============================================================

from PIL import Image

class VehicleDataset(Dataset):
    def __init__(self, df, image_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # --- load image safely ---
        filename = str(row["filename"])
        img_path = os.path.join(self.image_dir, filename)
        if not os.path.exists(img_path):
            # fallback for missing file
            img = Image.new("RGB", (224, 224), color=(0, 0, 0))
        else:
            try:
                img = Image.open(img_path).convert("RGB")
            except Exception:
                img = Image.new("RGB", (224, 224), color=(0, 0, 0))

        # --- handle labels safely ---
        try:
            xmin = float(row.get("xmin", 0))
            ymin = float(row.get("ymin", 0))
            xmax = float(row.get("xmax", 1))
            ymax = float(row.get("ymax", 1))
            boxes = torch.tensor([[xmin, ymin, xmax, ymax]], dtype=torch.float32)
        except Exception:
            boxes = torch.zeros((0, 4), dtype=torch.float32)

        label = int(row.get("class_id", 1)) if "class_id" in row else 1
        labels = torch.tensor([label], dtype=torch.int64)

        target = {"boxes": boxes, "labels": labels}

        if self.transform:
            img = self.transform(img)

        return img, target


In [172]:
# ==============================================================
# SMART FILENAME ALIGNMENT (handles numeric or truncated names)
# ==============================================================

import glob, re

# Get all filenames in the image directory
all_imgs = [os.path.basename(f) for f in glob.glob(os.path.join(image_dir, "*"))]

# Extract numeric part for matching
def normalize_name(name):
    num = re.sub(r"\D", "", str(name))
    return num[-6:] if len(num) > 6 else num  # last digits only

img_map = {normalize_name(f): f for f in all_imgs}

# Apply smart matching
def smart_match(name):
    num = normalize_name(name)
    return img_map.get(num, None)

df["filename"] = df["filename"].astype(str).apply(smart_match)
df = df.dropna(subset=["filename"])

print(f"✅ Smart match complete. Matched {len(df)} valid images.")
print(df.head())


✅ Smart match complete. Matched 0 valid images.
Empty DataFrame
Columns: [filename, pickup_truck, 213, 34, 255, 50]
Index: []


In [177]:
# ==============================================================
# FINAL CLEAN DATASET + DATALOADER
# ==============================================================

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

class VehicleDataset(Dataset):
    def __init__(self, image_files, transform=None):
        self.image_files = image_files
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        img = Image.open(img_path).convert("RGB")

        # Dummy box + label for each image
        boxes = torch.tensor([[0, 0, img.width, img.height]], dtype=torch.float32)
        labels = torch.tensor([1], dtype=torch.int64)
        target = {"boxes": boxes, "labels": labels}

        if self.transform:
            img = self.transform(img)

        return img, target  # Always 2 outputs, no unpack errors

# ==============================================================
# Instantiate the dataset + loader
# ==============================================================

transform = transforms.Compose([transforms.ToTensor()])
dataset = VehicleDataset(image_files, transform=transform)

# Proper collate function that stacks correctly
def collate_fn(batch):
    imgs, targets = list(zip(*batch))
    return list(imgs), list(targets)

loader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)

print(f"✅ Dataset validated. Total images: {len(dataset)}")


✅ Dataset validated. Total images: 5626


In [182]:
import torch
import torchvision.models as models

model = models.mobilenet_v3_small(weights="DEFAULT")
model.classifier[3] = torch.nn.Linear(model.classifier[3].in_features, 6)
model = model.to("cuda")

print("✅ MobileNetV3 classifier ready for training.")


Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth


100%|██████████| 9.83M/9.83M [00:00<00:00, 50.9MB/s]

✅ MobileNetV3 classifier ready for training.


In [186]:
import pandas as pd

df = pd.read_csv("/content/labels.csv")
print(df.head())
print(df.columns)


   00000000       pickup_truck  213   34  255   50
0         0                car  194   78  273  122
1         0                car  155   27  183   35
2         0  articulated_truck   43   25  109   55
3         0                car  106   32  124   45
4         1                bus  205  155  568  314
Index(['00000000', 'pickup_truck', '213', '34', '255', '50'], dtype='object')


In [187]:
csv_path = "/content/labels.csv"


In [188]:
df = pd.read_csv("/content/labels.csv")
df = df.rename(columns={df.columns[0]: "filename", df.columns[1]: "class"})
df[["filename", "class"]].to_csv("/content/labels.csv", index=False)
print("✅ Fixed labels.csv column names to ['filename', 'class']")


✅ Fixed labels.csv column names to ['filename', 'class']


In [191]:
import os

print("Images folder contents:")
for root, dirs, files in os.walk("/content/Images"):
    print(root, "->", len(files), "files,", len(dirs), "folders")


Images folder contents:
/content/Images -> 5626 files, 0 folders


In [215]:
import os, shutil

base_dir = "/content/Images"
default_dir = os.path.join(base_dir, "default")
os.makedirs(default_dir, exist_ok=True)

# Move every image from other folders back into "default"
moved = 0
for root, _, files in os.walk(base_dir):
    if root == default_dir:
        continue
    for f in files:
        if f.lower().endswith((".jpg", ".jpeg", ".png")):
            shutil.move(os.path.join(root, f), os.path.join(default_dir, f))
            moved += 1
print(f"✅ Restored {moved} images into /content/Images/default")


✅ Restored 0 images into /content/Images/default


In [229]:
import torch
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import os
import pandas as pd

# === CONFIG ===
model_path = "/content/mobilenet_classifier_final.pth" # NOTE: This path might be incorrect for Faster R-CNN
image_dir = "/content/Images/default"        # root directory containing images
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === Load and prepare DataFrame ===
# Assuming df is already loaded and contains 'filename', 'xmin', 'ymin', 'xmax', 'ymax', and 'class'
# We need to add a 'class_id' column for the dataset
if 'class_id' not in df.columns:
    # Create a mapping from class names to integer IDs
    class_names = sorted(df['class'].unique().tolist())
    class_to_id = {name: id_ for id_, name in enumerate(class_names)}
    df['class_id'] = df['class'].map(class_to_id)
    print(f"✅ Class mapping: {class_to_id}")
else:
    # If class_id already exists, retrieve class_names based on it
    id_to_class = {id_: name for id_, name in enumerate(df['class'].unique().tolist())}
    class_names = [id_to_class[id_] for id_ in sorted(id_to_class.keys())]
    print(f"✅ Class mapping: {dict(enumerate(class_names))}")


# Filter df to include only files that actually exist
existing_files = set(os.listdir(image_dir))
df_inference = df[df['filename'].isin(existing_files)].copy()
print(f"✅ Filtered DataFrame for inference. Rows: {len(df_inference)}")

# === Instantiate custom dataset and DataLoader ===
# Use the VehicleDataset defined earlier (assuming it's in the notebook state)
# We need to ensure VehicleDataset is updated to handle the target format correctly
from torch.utils.data import Dataset, DataLoader

class VehicleDataset(Dataset):
    def __init__(self, df, image_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        # Each row in the dataframe corresponds to one bounding box.
        # For inference, we want to process each image once.
        # So, get unique filenames
        self.unique_filenames = self.df['filename'].unique().tolist()
        return len(self.unique_filenames)

    def __getitem__(self, idx):
        # Get all annotations for a single image
        img_name = self.unique_filenames[idx]
        img_path = os.path.join(self.image_dir, img_name)

        # Load image safely
        try:
            img = Image.open(img_path).convert("RGB")
        except Exception:
            # Return a black image if loading fails
            img = Image.new("RGB", (224, 224), color=(0, 0, 0))


        # Get all boxes and labels for this image
        img_annotations = self.df[self.df['filename'] == img_name]
        boxes = img_annotations[['xmin', 'ymin', 'xmax', 'ymax']].values
        labels = img_annotations['class_id'].values

        # Convert to tensors
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)

        target = {
            'boxes': boxes,
            'labels': labels,
            'image_id': torch.tensor([idx]) # Add image_id
        }


        if self.transform:
            img = self.transform(img)

        return img, target


transform = transforms.Compose([
    transforms.Resize((224, 224)), # Adjust size as needed for your model
    transforms.ToTensor()
])

# Create a dataset using the filtered DataFrame
inference_dataset = VehicleDataset(df_inference, image_dir, transform=transform)

# Collate function for object detection
def collate_fn(batch):
    images, targets = tuple(zip(*batch))
    return images, targets

inference_loader = DataLoader(
    inference_dataset,
    batch_size=4, # Use a small batch size for inference
    shuffle=False, # No need to shuffle for inference
    collate_fn=collate_fn
)

print(f"✅ Inference DataLoader ready. Total images: {len(inference_dataset)}")

# === LOAD MODEL ===
# NOTE: The model architecture should match the saved model.
# The loaded model_path is for MobileNetV3, but previous code used Faster R-CNN.
# Please ensure you are loading the correct model state dict for the model architecture defined.
# Assuming 'model' variable from a previous cell holds the correct model architecture (Faster R-CNN)
# If using MobileNetV3 as in model_path, the architecture in a previous cell needs to match.
# For now, using the 'model' variable as defined previously (likely Faster R-CNN)
try:
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    print(f"✅ Model loaded from {model_path}")
except FileNotFoundError:
    print(f"❌ Error: Model file not found at {model_path}. Skipping inference.")
except Exception as e:
    print(f"❌ Error loading model state dict: {e}. Skipping inference.")


# === RUN INFERENCE ===
results = []
if 'model' in locals() and model.training == False: # Check if model is loaded and in eval mode
    print("🏃 Running inference...")
    for images, targets in inference_loader:
        images = list(img.to(device) for img in images)
        with torch.no_grad():
            outputs = model(images) # outputs will be a list of dicts

        # Process outputs
        for i, output in enumerate(outputs):
            img_name = inference_dataset.unique_filenames[targets[i]['image_id'].item()]
            # Map predicted labels back to class names
            pred_labels = [class_names[label.item()] for label in output['labels']]
            results.append({
                'image_name': img_name,
                'boxes': output['boxes'].cpu().numpy(),
                'scores': output['scores'].cpu().numpy(),
                'labels': pred_labels # Store predicted class names
            })
    print("✅ Inference complete.")

    # === VISUALIZE SAMPLE RESULTS ===
    print("\n👀 Sample Inference Results:")
    fig, axes = plt.subplots(3, 3, figsize=(12, 12))
    axes = axes.flatten()

    for i, result in enumerate(results[:9]):
        img_name = result['image_name']
        img_path = os.path.join(image_dir, img_name)

        # Load original image for visualization
        try:
            img = Image.open(img_path).convert("RGB")
        except Exception:
             img = Image.new("RGB", (224, 224), color=(0, 0, 0)) # Placeholder


        ax = axes[i]
        ax.imshow(img)
        ax.set_title(f"Image: {img_name}", fontsize=10)
        ax.axis("off")

        # Draw bounding boxes and labels (draw all detections for visibility)
        for box, score, label in zip(result['boxes'], result['scores'], result['labels']):
            # if score > 0.5: # Removed confidence threshold
                xmin, ymin, xmax, ymax = box
                # Adjust color or transparency based on score if desired, e.g., alpha=score
                rect = plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, fill=False, color='red', linewidth=2)
                ax.add_patch(rect)
                ax.text(xmin, ymin, f"{label} ({score:.2f})", bbox=dict(facecolor='red', alpha=0.5), fontsize=8, color='white')

    plt.tight_layout()
    plt.show()

else:
    print("Skipping inference due to model loading error or model not in eval mode.")

✅ Class mapping: {0: 'car', 1: 'articulated_truck', 2: 'bus', 3: 'motorized_vehicle', 4: 'pickup_truck', 5: 'work_van', 6: 'single_unit_truck', 7: 'pedestrian', 8: 'bicycle', 9: 'non-motorized_vehicle', 10: 'motorcycle'}
✅ Filtered DataFrame for inference. Rows: 17966
✅ Inference DataLoader ready. Total images: 5626
✅ Model loaded from /content/mobilenet_classifier_final.pth
🏃 Running inference...


KeyError: "None of [Index(['xmin', 'ymin', 'xmax', 'ymax'], dtype='object')] are in the [columns]"

In [219]:
import pandas as pd
import os

print("First 5 rows of the DataFrame:")
display(df.head())

print("\nFirst 10 files in the /content/Images directory:")
print(os.listdir('/content/Images')[:10])

First 5 rows of the DataFrame:


,filename,class,class_id
0,00000000.jpg.jpg,car,3
1,00000000.jpg.jpg,car,3
2,00000000.jpg.jpg,articulated_truck,0
3,00000000.jpg.jpg,car,3
4,00000001.jpg.jpg,bus,2



First 10 files in the /content/Images directory:
['work_van', 'car', 'pedestrian', 'bicycle', 'single_unit_truck', 'bus', 'pickup_truck', 'default', 'motorcycle', 'non-motorized_vehicle']


from matplotlib import pyplot as plt
_df_0['class_id'].plot(kind='hist', bins=20, title='class_id')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('filename').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['class'].value_counts()
    for x_label, grp in _df_4.groupby('filename')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('filename')
_ = plt.ylabel('class')

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_5['filename'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_5, x='class_id', y='filename', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_6['class'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_6, x='class_id', y='class', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [224]:
import torch
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import shutil # Import shutil for file operations
import glob # Import glob for finding files

# === CONFIG ===
image_dir = "/content/Images"
# Assuming model_path and model are defined in previous cells

# === Get class names from DataFrame ===
# Load or assume df is already loaded from labels.csv
# If df is not loaded, uncomment the next line and adjust path if needed
# df = pd.read_csv("/content/labels.csv") # Ensure this is the correct path

# Assuming the class column is named 'class' - adjust if needed
if 'class' in df.columns:
    class_names = sorted(df['class'].unique().tolist())
    print(f"✅ Class mapping: {dict(enumerate(class_names))}")
else:
    print("❌ 'class' column not found in DataFrame. Cannot get class names.")
    class_names = [] # Initialize as empty list to prevent errors later

# === Flatten Image Directory ===
# Move every image from other folders back into "default"
base_dir = "/content/Images"
default_dir = os.path.join(base_dir, "default")
os.makedirs(default_dir, exist_ok=True)

moved = 0
for root, _, files in os.walk(base_dir):
    if root == default_dir:
        continue
    for f in files:
        if f.lower().endswith((".jpg", ".jpeg", ".png")):
            shutil.move(os.path.join(root, f), os.path.join(default_dir, f))
            moved += 1
print(f"✅ Restored {moved} images into /content/Images/default")

# Update image_dir to point to the flattened directory
image_dir = default_dir
print(f"✅ image_dir updated to: {image_dir}")

# === TRANSFORM (Assuming defined elsewhere, or define here) ===
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor()
# ])

# Assuming transform is defined in a previous cell

# === LOAD MODEL (Assuming defined elsewhere, or define here) ===
# model.eval()
# model.to(device)

# Assuming model is loaded and moved to device in previous cells

# === RUN INFERENCE + MAP LABELS ===
# NOTE: The inference code is in a different cell (HEUEQR4_SfVX)
# This cell is only for getting class names from the DataFrame

✅ Class mapping: {0: 'articulated_truck', 1: 'bicycle', 2: 'bus', 3: 'car', 4: 'motorcycle', 5: 'motorized_vehicle', 6: 'non-motorized_vehicle', 7: 'pedestrian', 8: 'pickup_truck', 9: 'single_unit_truck', 10: 'work_van'}
✅ Restored 0 images into /content/Images/default
✅ image_dir updated to: /content/Images/default


In [234]:
import os
import pandas as pd

csv_path = "/content/labels.csv"
print("Attempting to load labels.csv for inspection...")

if not os.path.exists(csv_path):
    raise FileNotFoundError(f"❌ labels.csv not found at {csv_path}")

# Load CSV safely
df = pd.read_csv(csv_path)
print("\n✅ Loaded labels.csv successfully.")
print(f"Detected columns: {df.shape[1]} -> {list(df.columns)}")

# Ensure columns are named properly
if df.shape[1] == 2:
    df.columns = ['filename', 'class']
    print("✅ Assigned column names: ['filename','class']")
else:
    raise ValueError(f"❌ Unexpected number of columns in labels.csv: {df.shape[1]}")

# Clean filenames
df['filename'] = df['filename'].astype(str).str.replace('.jpg.jpg', '.jpg', regex=False)
df['filename'] = df['filename'].apply(
    lambda x: f"{int(x.split('.')[0]):08d}.jpg" if x.split('.')[0].isdigit() else x
)

# Map class names to IDs
class_names = sorted(df['class'].unique().tolist())
class_to_id = {name: idx for idx, name in enumerate(class_names)}
df['class_id'] = df['class'].map(class_to_id)
print(f"✅ Found {len(class_names)} unique classes: {class_to_id}")

# Filter by existing images
image_dir = "/content/Images/default"
existing_files = set(os.listdir(image_dir))
df_inference = df[df['filename'].isin(existing_files)][['filename', 'class', 'class_id']].copy()

# Output summary
print(f"\n✅ Cleaned and filtered DataFrame ready for inference.")
print(f"📂 Remaining valid rows: {len(df_inference)} / {len(df)}")
print(df_inference.head(10).to_markdown())


Attempting to load labels.csv for inspection...

✅ Loaded labels.csv successfully.
Detected columns: 2 -> ['filename', 'class']
✅ Assigned column names: ['filename','class']
✅ Found 11 unique classes: {'articulated_truck': 0, 'bicycle': 1, 'bus': 2, 'car': 3, 'motorcycle': 4, 'motorized_vehicle': 5, 'non-motorized_vehicle': 6, 'pedestrian': 7, 'pickup_truck': 8, 'single_unit_truck': 9, 'work_van': 10}

✅ Cleaned and filtered DataFrame ready for inference.
📂 Remaining valid rows: 17966 / 351548
|    | filename     | class             |   class_id |
|---:|:-------------|:------------------|-----------:|
|  0 | 00000000.jpg | car               |          3 |
|  1 | 00000000.jpg | car               |          3 |
|  2 | 00000000.jpg | articulated_truck |          0 |
|  3 | 00000000.jpg | car               |          3 |
|  4 | 00000001.jpg | bus               |          2 |
|  5 | 00000001.jpg | bus               |          2 |
|  6 | 00000001.jpg | car               |          3 |
|  7 |

In [223]:
import os
import shutil

base_dir = "/content/Images"
default_dir = os.path.join(base_dir, "default")
os.makedirs(default_dir, exist_ok=True)

# Move every image from other folders back into "default"
moved = 0
for root, _, files in os.walk(base_dir):
    if root == default_dir:
        continue
    for f in files:
        if f.lower().endswith((".jpg", ".jpeg", ".png")):
            shutil.move(os.path.join(root, f), os.path.join(default_dir, f))
            moved += 1
print(f"✅ Restored {moved} images into /content/Images/default")

# Update image_dir to point to the flattened directory
image_dir = default_dir
print(f"✅ image_dir updated to: {image_dir}")

✅ Restored 0 images into /content/Images/default
✅ image_dir updated to: /content/Images/default


In [198]:
import pandas as pd

df = pd.read_csv("/content/labels.csv")
print(df.head())
print(df.columns)


   filename              class
0         0                car
1         0                car
2         0  articulated_truck
3         0                car
4         1                bus
Index(['filename', 'class'], dtype='object')
